In [ ]:
import h5py as h5
import numpy as np
import json
import matplotlib.pyplot as plt
import photodiag
from scipy.optimize import curve_fit

%matplotlib notebook

In [ ]:
def LoadData(datafn):
    with h5.File(datafn, 'r') as f:
        pulseIDs = f['pulse_id'][:]
        TOF =-f['SAROP11-PALMK118:CH1_BUFFER/data'][:]
        indexPumped = pulseIDs%2 == 0
    return TOF, pulseIDs
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-((x-mu)**2/(2.*sigma**2)))
def PrepData(DataIn):
    BG = DataIn[0:100].mean()
    return DataIn-BG

In [ ]:
# setup PALM Process
palm = photodiag.PalmSetup(
    channels={'0': 'SAROP11-PALMK118:CH1_BUFFER', '1': 'SAROP11-PALMK118:CH2_BUFFER'},
#     noise_range=[0, 250], energy_range=np.linspace(5000, 5700, 1000), # @ 9 kev
    noise_range=[0, 250], energy_range=np.linspace(500, 2500, 1000), # @ 8 kev looking at 2s in Xe
    
#     noise_range=[0, 250], energy_range=np.linspace(0, 5150, 1000),   
)
# palm.load_etof_calib(f'/sf/alvra/data/p17589/res/PhotoDiag/scan_info/2018-10-24_12:12:13.palm_etof')
palm.load_etof_calib(f'/sf/alvra/data/p17589/res/PhotoDiag/scan_info/2019-03-25_11:53:57.palm_etof')


In [ ]:
# Load data
fn = '/sf/alvra/data/p17589/res/PhotoDiag/scan_info/PALM_resol_205_scan_info.json'

with open(fn) as f:
    data = json.load(f)
numFiles = len(data['scan_files'])
LensVolt = data['scan_values']
# pulse_ids = f['data/SLAAR11-PALMK118:CH2_BUFFER/pulse_id'][:]

TOF = []
for i in range(0,numFiles):
    (TOFtmp, pulseIDstmp) = LoadData(str(data['scan_files'][i][0]))
    TOF.append(TOFtmp)
TOF = np.asarray(TOF)
TOF = TOF.mean(axis=1)

streaked = 'SAROP11-PALMK118:CH1_BUFFER'
reference = 'SAROP11-PALMK118:CH1_BUFFER'

In [ ]:
# PALM Proc
eVInt =[]
# eVInt2 =[]

eV = palm.energy_range
for i in range(0, TOF.shape[0]):
    tmp = TOF[i,:]
    delays, pulse_lengths, (input_data, lags, corr_res_uncut, corr_results)= palm.process({'0': tmp[np.newaxis,:], '1': tmp[np.newaxis,:]}, debug=True, noise_thr=0, jacobian=False, peak='max')
    eVInt.append(input_data['0'][0,:])
eVInt = np.asarray(eVInt)

In [ ]:
plt.figure()
plt.plot(palm.energy_range,eVInt[2,:])
# plt.pcolormesh(eVInt)
# plt.xlim([0,1000])

In [ ]:
LensVoltnp = np.asarray(LensVolt)

In [ ]:
plt.figure()
plt.pcolormesh(LensVoltnp[:,1], palm.energy_range,np.transpose(eVInt))
# plt.ylim([1000, 2500])
plt.xlabel('Lens voltage')
plt.ylabel('Electron KE')

In [ ]:
# Fit spectra
p0 = [0.1, 1250,20]
Coeff = []
for i in range(0, TOF.shape[0]):
    try:
        coeff, var_matrix = curve_fit(gauss, eV, PrepData(eVInt[i,:]), p0=p0)
        Coeff.append(coeff)
    except:
        Coeff.append([0,0,0])
Coeff = np.asarray(Coeff)

In [ ]:
fig=plt.figure()
plt.subplot(2,1,1)
plt.title(fn[46:])
plt.plot(LensVolt, np.abs(Coeff[:,2]))
# plt.ylim([10,25])
plt.xlabel('Lens voltage')
plt.ylabel('sigma [eV]')
plt.subplot(2,1,2)
plt.plot(LensVolt, np.abs(Coeff[:,0]))
plt.xlabel('Lens voltage')
plt.ylabel('Int [arb]')
plt.tight_layout()
fig.set_size_inches(18.5, 10.5)
# fig.savefig('/das/work/units/alvra/p17569/PhotoDiag/figures/'+fn[46:60]+'.png')


In [ ]:
plt.figure()
plt.plot(palm.energy_range, eVInt[-1,:])
plt.plot(eV, gauss(eV, *Coeff[-1,:]))
plt.xlim([5400, 5500])

In [ ]:
plt.figure()
plt.pcolormesh(eV, LensVolt, eVInt)
# plt.xlim([1200, 1800])